In [1]:
import zmq
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time

import asyncio


path = "database/"

def database(Id , interval):
    engine = sqlalchemy.create_engine('sqlite:///'+path+'database_{}.db'.format(interval))
    
    data = pd.read_sql(Id+'USDT' ,engine)
    data.set_index('time' , inplace=True)
    data['volume'] = pd.to_numeric(data['volume'])
    data = data[['open', 'high', 'low' , 'close' , 'volume', 'symbol', 'close_time']]
    return data


begin = '2023'
end = '2023'
interval = "1d"

SYMBOL_1 = "BTC"
SYMBOL_2 = "ETH"
SYMBOL_3 = "SOL"

data_1 = database(SYMBOL_1, interval)
data_1 = data_1.loc[begin:end]

data_2 = database(SYMBOL_2, interval)
data_2 = data_2.loc[begin:end]

data_3 = database(SYMBOL_3, interval)
data_3 = data_3.loc[begin:end]

In [2]:

class Server_data():
    
    def __init__(self, data_1, data_2, data_3, sleep =2):
        self.data_1 = data_1
        self.data_2 = data_2
        self.data_3 = data_3
        
        self.bar = 0
        self.genere_data(sleep)
    
    
    def to_message(self, data, i, stop = 'false'):
        date = data.index[i]
        symbol = data['symbol'][0]
        date = date.timestamp()
        msg = '{} {} {} {} {} {} {} {}'.format(symbol, date,
                                    data.iloc[i]['open'], data.iloc[i]['high'],
                                    data.iloc[i]['low'], data.iloc[i]['close'],
                                    data.iloc[i]['volume'], stop
                                    )
        return msg
    
    
    
    def genere_data(self, sleep):
        context = zmq.Context()
        socket = context.socket(zmq.PUB)
        socket.bind('tcp://127.0.0.1:8080')
        
        while True:
            try:
                msg_1 = self.to_message(self.data_1, self.bar)
                msg_2 = self.to_message(self.data_2, self.bar)
                msg_3 = self.to_message(self.data_3, self.bar)
            except:
                msg = " ... end ... "
                print(msg)
                break
            
            print(msg_1)
            print(msg_2)
            print(msg_3)
            socket.send_string(msg_1)
            socket.send_string(msg_2)
            socket.send_string(msg_3)
            
            self.bar += 1
            time.sleep(sleep)
            
            
server = Server_data(data_1, data_2, data_3, sleep = 2)


BTCUSDT 1672531200.0 16541.77 16628.0 16499.01 16616.75 96925.41374 false
ETHUSDT 1672531200.0 1196.13 1204.67 1190.57 1200.34 117762.0685 false
SOLUSDT 1672531200.0 9.97 10.09 9.69 9.99 2287512.63 false
BTCUSDT 1672617600.0 16617.17 16799.23 16548.7 16672.87 121888.57191 false
ETHUSDT 1672617600.0 1200.33 1224.29 1193.0 1214.0 252794.14 false
SOLUSDT 1672617600.0 9.99 12.0 9.81 11.28 7628264.93 false
BTCUSDT 1672704000.0 16672.78 16778.4 16605.28 16675.18 159541.53733 false
ETHUSDT 1672704000.0 1213.99 1219.89 1204.93 1214.55 205519.4944 false
SOLUSDT 1672704000.0 11.28 13.91 11.0 13.37 14986491.36 false
BTCUSDT 1672790400.0 16675.65 16991.87 16652.66 16850.36 220362.18862 false
ETHUSDT 1672790400.0 1214.56 1272.6 1212.71 1256.9 457799.6648 false
SOLUSDT 1672790400.0 13.37 14.24 12.83 13.43 12612735.49 false
BTCUSDT 1672876800.0 16850.36 16879.82 16753.0 16831.85 163473.56641 false
ETHUSDT 1672876800.0 1256.91 1259.95 1242.81 1251.24 272805.33 false
SOLUSDT 1672876800.0 13.43 13.89 12